In [2]:
from utils.dataLoader import DataLoader
from utils.modelLoader import ModelLoader
from utils.SiameseModel import SiameseModel
import tensorflow as tf
import pandas as pd

In [20]:
def loadImage(path: str):
    image_size = (224, 224)
    image_channels = 3
    img = tf.io.read_file(path)
    img = tf.image.decode_image(
        img, channels=image_channels, expand_animations=False
    )
    img = tf.image.central_crop(img, 0.7)
    img = tf.image.resize(img, image_size, method="bilinear")
    img.set_shape((image_size[0], image_size[1], image_channels))
    return img

def loadImages(path1:str, path2:str):
    return (
        loadImage(path1),
        loadImage(path2)
    )



In [30]:
data = [
    ["../data/img_align_celeba/000791.jpg","../data/img_align_celeba/012203.jpg"],
    ["../data/img_align_celeba/000791.jpg", "../data/img_align_celeba/017522.jpg"],
    ["../data/img_align_celeba/002345.jpg", "../data/img_align_celeba/014529.jpg"],
    ["../data/img_align_celeba/000791.jpg", "../data/img_align_celeba/014529.jpg"],
    ["../data/img_align_celeba/002345.jpg", "../data/img_align_celeba/000791.jpg"],
    ["../data/img_align_celeba/017522.jpg", "../data/img_align_celeba/012203.jpg"]
]
metaData = pd.DataFrame(data, columns=["anchor", "compare"])

anchor_images = metaData["anchor"]
compare_images = metaData["compare"]

anchor_dataset = tf.data.Dataset.from_tensor_slices(anchor_images)
positive_dataset = tf.data.Dataset.from_tensor_slices(compare_images)

dataset = tf.data.Dataset.zip((anchor_dataset, positive_dataset))
dataset = dataset.map(loadImages)

dataset = dataset.batch(1)
dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

embedding: tf.keras.Model = tf.keras.models.load_model("../models/Siamese_MobileNetV1_Train5000_All_embedding")

for sample in iter(dataset):
    anchor, compare = sample

    anchor_embedding, compare_embedding = (
        embedding(tf.keras.applications.mobilenet.preprocess_input(anchor)),
        embedding(tf.keras.applications.mobilenet.preprocess_input(compare)),
    )
    
    cosine_similarity = tf.keras.metrics.CosineSimilarity()
    positive_similarity = cosine_similarity(anchor_embedding, compare_embedding)

    print("Similarity:", positive_similarity.numpy())

Similarity: 0.93398213
Similarity: 0.97547746
Similarity: 0.89644086
Similarity: 0.9726523
Similarity: 0.8594779
Similarity: 0.9604407


In [ ]:
embedding: tf.keras.Model = tf.keras.models.load_model("../models/Siamese_MobileNetV1_Train5000_All_embedding")

anchor = loadImage("../data/img_align_celeba/000791.jpg")
compare = loadImage("../data/img_align_celeba/012203.jpg")

anchor_embedding, compare_embedding = (
    embedding.predict(tf.keras.applications.mobilenet.preprocess_input(anchor)),
    embedding.predict(tf.keras.applications.mobilenet.preprocess_input(compare)),
)

cosine_similarity = tf.keras.metrics.CosineSimilarity()
positive_similarity = cosine_similarity(anchor_embedding, compare_embedding)

print("Similarity:", positive_similarity.numpy())

ValueError: Tensor's shape (224, 224, 3) is not compatible with supplied shape (None, 224, 224, 3).